# Heavyweight Hybrid: AerSimulator Classical Hardware

This notebook implements the **Heavyweight Hybrid** approach to reduced portfolio optimization using **AerSimulator classical hardware (CPU)**.

- **Problem Size**: 10 bonds (reduced from 31)
- **Ansatz**: TwoLocal with full entanglement + Local Search post-processing
- **Parameters**: Low reps (1-2) + Local Search optimization
- **Quantum Measurements**: 512 shots per epoch
- **Quantum Epochs**: 2 epochs of CVaR-VQE
- **Local Search Epochs**: 8 epochs of classical refinement
- **CVaR Parameter**: α = 0.1 (focuses on worst 10% of measurements)
- **Hardware**: AerSimulator classical device

## Setup and Imports

In [1]:
import os
import sys
import time
import numpy as np
import pickle
from pathlib import Path
from dataclasses import asdict
import docplex.mp.model_reader

# add root directory to path
ROOT = Path(os.getcwd()).parent
sys.path.append(str(ROOT))

# quantum imports
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# project imports
# 1. LP/QUBO processing
from src.sbo.src.utils.lp_utils import load_quadratic_program
from src.sbo.src._problems.substitute_variables import substitute_variables
from src.sbo.src._converters.quadratic_program_to_qubo import QuadraticProgramToQubo
# 2. ansatz building
from src.step_1 import build_ansatz
# 3. quantum optimization
from src.sbo.src.patterns.building_blocks.step_3 import HardwareExecutor
# 4. local search
from src.step_1 import model_to_obj # same as importing model_to_obj_sparse_numba
from src.sbo.src.optimizer.local_search import repeated_local_search_general
# 5. experiment saving
from src.experiment import Experiment

print(f"Working directory: {ROOT}")
print(f"Python path includes: {str(ROOT)} in sys.path")

Working directory: C:\Users\ASUS\Downloads\WISER_Optimization_VG-main
Python path includes: C:\Users\ASUS\Downloads\WISER_Optimization_VG-main in sys.path


## Problem Setup: 31-Bond Portfolio Optimization

The problem starts as an LP file, which is converted into a quadratic function and then transformed into a Quadratic Unconstrained Binary Optimization (QUBO) problem using predefined modules.

In [2]:
# load the 31-bond problem
lp_file_path = str(ROOT) + '/data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp'

# convert LP file into quadratic problem
quadratic_problem = load_quadratic_program(lp_file_path)

# convert quadratic program into QUBO
converter = QuadraticProgramToQubo()
qubo = converter.convert(quadratic_problem)

print(f"31-bond problem: {quadratic_problem.get_num_binary_vars()} binary variables")
print(f"QUBO variables: {qubo.get_num_vars()}")
print(f"QUBO objective sense: {qubo.objective.sense}")
print(f"QUBO constant term: {qubo.objective.constant}")

31-bond problem: 31 binary variables
QUBO variables: 31
QUBO objective sense: ObjSense.MINIMIZE
QUBO constant term: 0.0


## Problem Size Reduction

Due to IBM Quantum's free tier limitations (10 minutes of QPU access per month as of 2025), the problem size is reduced to make it feasible for execution on real quantum hardware. The `substitute_variables()` function assigns a constant value of 0 to a range of variables out of 31 total variables. This effectively reduces the problem dimensionality while maintaining the optimization structure.

In this implementation, 10 bonds are kept for demonstration purposes; however, this number is configurable via the `NUM_BONDS_TO_KEEP` parameter and can be adjusted based on available computational resources and desired problem complexity.

- **Original Problem**: 31 bonds (31 qubits required)
- **Reduced Problem**: 10 bonds (10 qubits required)

In [3]:
# set the number of bonds to keep
NUM_BONDS_TO_KEEP = 10  # -- MODIFY HERE --

# get all variable names
var_names = [var.name for var in quadratic_problem.variables]
total_vars = len(var_names)
print(f"Total variables: {total_vars}")
print()
print(f"Keeping first {NUM_BONDS_TO_KEEP}: {var_names[:NUM_BONDS_TO_KEEP]}")
print()
print(f"Discarding remaining {total_vars - NUM_BONDS_TO_KEEP}: {var_names[NUM_BONDS_TO_KEEP:]}")
print()

# reduce problem size by fixing unused variables to 0
constants = {var_names[i]: 0 for i in range(NUM_BONDS_TO_KEEP, total_vars)}
smaller_quadratic_problem = substitute_variables(quadratic_problem, constants=constants)
smaller_qubo = converter.convert(smaller_quadratic_problem)

print(f"{NUM_BONDS_TO_KEEP}-bond problem: {smaller_quadratic_problem.get_num_binary_vars()} binary variables")
print(f"QUBO variables: {smaller_qubo.get_num_vars()}")
print(f"QUBO objective sense: {smaller_qubo.objective.sense}")
print(f"QUBO constant term: {smaller_qubo.objective.constant}")

Total variables: 31

Keeping first 10: ['iTrade_444859BY7', 'iTrade_45687VAB2', 'iTrade_15135BAW1', 'iTrade_081437AT2', 'iTrade_444859BR2', 'iTrade_443201AC2', 'iTrade_760759BA7', 'iTrade_026874DS3', 'iTrade_14448CBC7', 'iTrade_36166NAK9']

Discarding remaining 21: ['iTrade_539830CD9', 'iTrade_760759BC3', 'iTrade_907818FX1', 'iTrade_24422EWZ8', 'iTrade_444859BV3', 'iTrade_13645RAD6', 'iTrade_24422EXP9', 'iTrade_91324PEJ7', 'iTrade_56501RAN6', 'iTrade_75513EAD3', 'iTrade_438516CM6', 'iTrade_314353AA1', 'iTrade_444859CA8', 'iTrade_020002BJ9', 'iTrade_759351AP4', 'iTrade_655844CT3', 'iTrade_540424AT5', 'iTrade_097023CJ2', 'iTrade_13645RBF0', 'iTrade_655844CR7', 'iTrade_21871XAS8']

10-bond problem: 10 binary variables
QUBO variables: 10
QUBO objective sense: ObjSense.MINIMIZE
QUBO constant term: 0.0


## Objective Function

The objective function is given by the formula:

$$
\min \sum_{\ell \in L} \sum_{j \in J} \rho_j \left( \sum_{c \in K_\ell} \beta_{c,j} \, x_c - K_{\ell,j}^{\text{target}} \right)^2
$$

The goal is to minimize the difference between a portfolio's characteristics and their target values, ensuring the portfolio matches the targets as closely as possible. This objective function is handled internally by the QUBO formulation.

In [4]:
# create objective function
def objective_function(x):
    """Objective function for the reduced QUBO problem"""
    return smaller_qubo.objective.evaluate(x)  # uses smaller_qubo instead

## Backend Configuration

Using **classical hardware** for noiseless quantum **simulation**, the backend is the AerSimulator, a concrete implementation of BackendV2, the abstract interface/protocol for all quantum backends in Qiskit.

In [5]:
num_vars = smaller_qubo.get_num_vars()

# create backend simulator (AerSimulator)
aer_options = {'method': 'matrix_product_state', 'n_qubits': num_vars}
backend = AerSimulator(**aer_options)

print(f"Backend: {backend}")
print(f"Number of qubits (backend): {backend.num_qubits}")
print(f"Number of qubits (problem): {num_vars}")

Backend: AerSimulator('aer_simulator_matrix_product_state')
Number of qubits (backend): 10
Number of qubits (problem): 10


## Ansatz Configuration

In [6]:
# TwoLocal configuration
ansatz = 'TwoLocal'
ansatz_params = {
    'reps': 2,  # low reps for shallow circuits
    'entanglement': 'full',  # full entanglement for maximum connectivity
    'rotation_blocks': 'ry',  # rotation gates
    'entanglement_blocks': 'cz',  # controlled-Z entangling gates
    'insert_barriers': False,  # no barriers for efficiency
}

print(f"Configuration: {ansatz} with {ansatz_params}")
print(f"Expected: Parameter-heavy with shallow circuits")

Configuration: TwoLocal with {'reps': 2, 'entanglement': 'full', 'rotation_blocks': 'ry', 'entanglement_blocks': 'cz', 'insert_barriers': False}
Expected: Parameter-heavy with shallow circuits


## Ansatz Building

In [7]:
# build TwoLocal ansatz
ansatz_circuit, initial_layout = build_ansatz(
    ansatz=ansatz,
    ansatz_params=ansatz_params,
    num_qubits=num_vars,
    backend=backend
)

print(f"Raw TwoLocal Ansatz has {ansatz_circuit.num_parameters} parameters")
print(f"Circuit depth: {ansatz_circuit.depth()}")
print(f"Number of gates: {len(ansatz_circuit.data)}")
print(f"Entanglement: {ansatz_params['entanglement']}")
print(f"Repetitions: {ansatz_params['reps']}")

Raw TwoLocal Ansatz has 30 parameters
Circuit depth: 2
Number of gates: 12
Entanglement: full
Repetitions: 2


## Ansatz Transpilation

In [8]:
# transpile the ansatz for the target backend
print("Transpiling ansatz for target backend...")
isa_ansatz = generate_preset_pass_manager(target=backend.target, optimization_level=3, initial_layout=initial_layout).run(ansatz_circuit)

print(f"Transpiled ansatz has {isa_ansatz.num_parameters} parameters")
print(f"Transpiled circuit depth: {isa_ansatz.depth()}")
print(f"Transpiled gates: {len(isa_ansatz.data)}")

Transpiling ansatz for target backend...
Transpiled ansatz has 30 parameters
Transpiled circuit depth: 31
Transpiled gates: 131


## Conditional Value at Risk (CVaR)

CVaR is a generalization of both the sample mean and the minimum. It represents the expected value of the lowest α-fraction (i.e., the lower tail) of a distribution. In this case, the distribution of sampled energy values from a quantum Hamiltonian.

According to Barkoutsos et al. (2020), to compute CVaR:

- The Hamiltonian is sampled (measured indirectly) *K* times.
- Each sample yields an energy value.
- The energy values are sorted in ascending order (lowest to highest).
- CVaR is then calculated as the average of the lowest ⌈αK⌉ values using the formula:

$$
\text{CVaR}_\alpha = \frac{1}{\lceil \alpha K \rceil} \sum_{k=1}^{\lceil \alpha K \rceil} H_k
$$

Here, $H_k$ denotes the k-th lowest observed energy value.

In implementation:

- The CVaR calculation is performed by the function <code>calc_cvar()</code> in <code>"src/sbo/src/patterns/building_blocks/step3.py"</code>.
- Sampled measurement results are mapped to energy values by the function <code>_sampler_result_to_cvar()</code> in <code>"src/sbo/src/patterns/building_blocks/step3.py"</code>.

## Standard VQE

After translating a problem into QUBO form and constructing a Hamiltonian for an n-qubit system, the standard VQE algorithm samples (i.e., indirectly measures) the Hamiltonian *K* times. These measurement outcomes are summed and averaged to estimate the expectation value $\langle \psi(\theta) | H | \psi(\theta) \rangle$. The lowest expectation value, corresponding to a specific set of parameters in the chosen ansatz that minimizes the cost function, is taken as the optimal solution (bitstring) with high probability.

## CVaR-VQE

In contrast, CVaR-VQE optimizes a CVaR objective, focusing on the average of the worst $\alpha$-fraction of sampled energies. The minimal CVaR value, corresponding to certain ansatz parameters minimizing this objective, is considered the optimal solution (bitstring).

## Hardware Executor

The HardwareExecutor orchestrates the entire VQA process. It initializes quantum circuit parameters using π/3 (≈1.047) as a reference value, which provides a good starting point for optimization. The CVaR parameters are set to focus on the worst 10% of measurements (alpha=0.1) with 512 quantum measurements per epoch.

The executor uses the NFT (Natural Frequency Tuning) optimizer with the following configuration:
- **objective_fun**: QUBO objective function to minimize
- **backend**: Quantum simulator backend (AerSimulator)
- **isa_ansatz**: TwoLocal quantum circuit ansatz
- **optimizer_theta0**: Initial parameter values
- **optimizer_method**: NFT optimizer
- **sampler_options**: Quantum measurement settings
- **solver_options**: Maximum 2 epochs, CVaR parameter, random parameter updates

In [9]:
# initialize quantum circuit parameters
theta_initial = np.pi/3 * np.ones(isa_ansatz.num_parameters)

# cvar optimization parameters
alpha = 0.1  # cvar parameter: focus on worst 10% of measurements # -- MODIFY HERE --
shots = 512  # number of quantum measurements per epoch # -- MODIFY HERE --

# create HardwareExecutor for quantum optimization
he = HardwareExecutor(
    objective_fun=objective_function,
    backend=backend,
    isa_ansatz=isa_ansatz,  # use transpiled ansatz
    optimizer_theta0=theta_initial,
    optimizer_method="nft",
    refvalue=None,
    sampler_options={'default_shots': shots},
    use_session=False,
    verbose="progress",
    file_name=None,
    store_all_x=True,
    solver_options={
        "max_epoch": 2,  # -- MODIFY HERE --
        "alpha": alpha,
        "random_update": True
    }
)

## Simulation

The TwoLocal VQA optimization with CVaR executes using AerSimulator, followed by local search refinement. The process involves:

1. Initialization: Quantum circuit parameters are set using π/3 as reference values
2. Iteration Loop: For each optimization iteration:
   - Prepare quantum state using TwoLocal ansatz
   - Measure Hamiltonian expectation value K times (shots)
   - Calculate CVaR from the worst α-fraction of measurements
   - Update parameters using NFT optimizer
3. Convergence: Continue until convergence or max_epoch limit
4. Local Search Phase: Apply classical local search to refine the best quantum solution
5. Final Solution: Extract the best solution from the hybrid approach

**Timeout Handling:** Execution time is monitored to track performance and ensure compliance with resource constraints (QPU time limits for real hardware, or performance metrics for simulators).

**Partial Results:** If execution is interrupted, the partial solution remains valid and analyzable. The `HardwareExecutor` class captures the best solution found before interruption, ensuring incomplete runs produce usable results for comparison and analysis.

The optimization extracts the best solution found, which includes:
- **best_x**: Binary vector representing selected bonds (1=selected, 0=not selected)
- **best_fx**: Objective function value (CVaR) of the best solution

In [10]:
# run the TwoLocal VQA optimization with cvar
print("Starting TwoLocal VQA optimization with CVaR...")

# record start time for performance measurement
start_time = time.time()

# set maximum time limit
max_time_seconds = 9 * 60 # 9 minutes to leave 1 min buffer

# execute the variational quantum optimization
result = he.run()

# check if execution exceeded maximum time limit
if time.time() - start_time > max_time_seconds:
    print("Warning: Execution exceeded 10-minute QPU time limit!")

# record end time and calculate total optimization time
end_time = time.time()
print(f"Quantum optimization ended in {end_time - start_time:.2f} seconds")

# extract the best solution found during quantum optimization
best_x_quantum = he.optimization_monitor.objective_monitor.best_x
best_fx_quantum = he.optimization_monitor.objective_monitor.best_fx

print(f"Quantum best solution: {best_x_quantum}")
print(f"Quantum best objective value: {best_fx_quantum}")

Starting TwoLocal VQA optimization with CVaR...


2025-10-16 23:56:23,533 INFO optimization_wrapper: run...
2025-10-16 23:56:23,536 DEBUG optimization_wrapper: Using X0 [1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755]
2025-10-16 23:56:23,537 INFO nft: Using parameters: max_epoch 2
2025-10-16 23:56:23,538 INFO nft: Using parameters: random_update True
2025-10-16 23:56:23,539 INFO nft: Using parameters: epoch_start 0
2025-10-16 23:56:23,541 INFO nft: Using parameters: idx_set_start None
2025-10-16 23:56:23,542 INFO nft: Using parameters: step_start 0
2025-10-16 23:56:24,557 INFO optimization_monitor: optimizer internal status: () {'idx_set': array([ 3, 21, 17, 24,  0, 14, 15, 29,  8,  4, 12,  1, 23, 25,  7, 10,  2,
       20, 13, 16, 26, 27, 19,  6,  9, 1

Quantum optimization ended in 24.34 seconds
Quantum best solution: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Quantum best objective value: -1444.0188979272036


## Post-Processing: Local Search

The local search phase applies classical optimization to refine the best quantum solution.

## Local Search Configuration

The `repeated_local_search_general()` function takes these parameters:
- **x**: Quantum best solution
- **val**: Quantum best objective value
- **func**: Numba-compatible objective function
- **options**: Configuration dictionary

The first two parameters are already obtained, while the remaining two must be set prior to running local search.

### Func: Numba-Compatible Objective Function

Since local search uses numba-compiled code for speed, the CPLEX-parsable LP file must be converted into a docplex model. This model is then passed to the `model_to_obj()` function to generate a numba-compatible objective function. The previously loaded `quadratic_problem` (created from the same LP file) cannot be used because it produces a `QuadraticProgram` object, which is not compatible with the `model_to_obj()` function, as it expects a docplex model.

- LP file → QuadraticProgram → objective function is **not numba-compatible** → usable for quantum phase, **not usable for local search**
- LP file → docplex model → objective function is **numba-compatible** → **usable for local search**

The local search objective function is built from the full 31-variable LP file but automatically handles variable-length input arrays. NumPy's broadcasting behavior allows the function to work with reduced solutions by treating missing elements as zeros, ensuring compatibility between the reduced quantum solution and the full-problem objective function.

In [11]:
# create a docplex Model object
docplex_model = docplex.mp.model_reader.ModelReader.read(lp_file_path)

# decorate the objective function to be numba compatible
numba_compatiable_objective_function = model_to_obj(docplex_model)

31 15


### Options: Configuration Dictionary

- **num_bitflips**: Number of bits to flip per iteration (set to 1 for fine-tuning).
- **maxepoch**: Maximum number of local search epochs (set to 8). This complements the 2 epochs from the quantum phase, resulting in a total of 10 epochs in the hybrid approach.
- **maxfevals_per_variable**: Maximum number of times the objective function is evaluated per variable during one epoch of local search (set to 3 for a balance between solution quality and speed). For example, with 10 variables and this parameter set to 3, the algorithm performs up to 30 evaluations per epoch.
- **refval**: Must be a float (not None) to avoid typing errors. Local search stops when the current value is close to or better than this reference value (set to infinity to disable early stopping).

In [12]:
# create doe_localsearch pattern
doe_ls = {
    'local_search_doe': 'balance',
    'local_search_num_bitflips': 1,
    'local_search_maxiter': None,
    'local_search_maxepoch': 8,  # -- MODIFY HERE --
    'local_search_maxfevals_per_variable': 3
}

# calculate maxfevals based on number of variables
if 'local_search_maxfevals_per_variable' in doe_ls:
    doe_ls['local_search_maxfevals'] = len(best_x_quantum) * doe_ls.pop('local_search_maxfevals_per_variable')

# define refval # cannot be none
refval = float('inf')

In [13]:
# apply local search post-processing
print("Starting local search refinement...")

# record start time for performance measurement
local_search_start = time.time()

# call local search
best_x_hybrid, best_fx_hybrid, num_epochs, num_fevals, vals = repeated_local_search_general(
    x=best_x_quantum,
    val=best_fx_quantum,
    func=numba_compatiable_objective_function,
    options=doe_ls | {'refval': refval}
)

# create result dictionary to match postprocess format
refined_solution = {
    'status': 'success',
    'objective': float(best_fx_hybrid),
    'solution': best_x_hybrid,
    'execution_callback_count': he.optimization_monitor.callback_count,
    'execution_results': he.optimization_monitor.list_callback_res,
    'execution_f_value_best': he.optimization_monitor.list_callback_monitor_best,
    'message': "Local search completed successfully"
}

# record end time and calculate total local search time
local_search_end = time.time()
print(f"Local search completed in {local_search_end - local_search_start:.2f} seconds")
print(f"Hybrid best solution: {best_x_hybrid}")
print(f"Hybrid best objective value: {best_fx_hybrid}")
print(f"Improvement: {best_fx_quantum - best_fx_hybrid:.6f}")

Starting local search refinement...
Local search completed in 5.22 seconds
Hybrid best solution: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Hybrid best objective value: -1444.0188979272036
Improvement: 0.000000


## Result

The experiment is saved as a pickle file containing all the necessary data for analysis, reproducibility, debugging, and research. The saved data includes optimization results, convergence data, performance metrics, configuration settings, and experiment metadata.

The experiment object includes:
- Configuration: Ansatz, parameters, optimizer settings, CVaR alpha, local search settings
- Performance: Execution time, iterations, convergence behavior
- Results: Best solution, objective value, relative gap
- Monitoring: Parameter evolution, objective function calls, optimization trajectory
- Hardware: Backend, shots, classical hardware specs

In [ ]:
# set path
experiment_path = Path(str(ROOT) + f"/project/results/exp_heavyweight_hybrid_cpu_reduced_{num_vars}_bonds.pkl")

In [15]:
# create an Experiment object from the hybrid results
# use from_step3()
experiment = Experiment.from_step3(
    experiment_id="heavyweight_hybrid_run",
    ansatz="TwoLocal",
    ansatz_params={'reps': 2, 'entanglement': 'full'},
    theta_initial="piby3",
    device="AerSimulator",  # use AerSimulator instead
    optimizer="nft",
    alpha=alpha,
    theta_threshold=None,
    lp_file="31bonds/docplex-bin-avgonly-nocplexvars.lp",
    shots=shots,
    refx=None,      # set this for a reference solution
    refvalue=None,  # set this for a reference value
    classical_hw=Experiment.get_current_classical_hw(),
    step3_time=end_time - start_time,
    step3_job_ids=he.job_ids,
    result=result,
    optimization_monitor=he.optimization_monitor
)

# override step4 fields that were set to None in from_step3()
experiment.step4_time = local_search_end - local_search_start
experiment.step4_result_best_x = best_x_hybrid
experiment.step4_result_best_fx = best_fx_hybrid
experiment.step4_iter_best_fx = refined_solution.get('execution_f_value_best', [])

# save the experiment for analysis
experiment_path.parent.mkdir(parents=True, exist_ok=True)
with open(experiment_path, 'wb') as f:
    pickle.dump(asdict(experiment), f)